## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-28-16-56-19 +0000,bbc,Five things that could get more expensive for ...,https://www.bbc.com/news/articles/cvgpvq20qmdo...
1,2025-07-28-16-54-17 +0000,nypost,Reversal of Etan Patz murder conviction raises...,https://nypost.com/2025/07/28/us-news/reversal...
2,2025-07-28-16-54-00 +0000,wsj,Stock Market Today: Euro Drops After Trump Str...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-07-28-16-54-00 +0000,nyt,Trump Administration Live Updates: President D...,https://www.nytimes.com/live/2025/07/28/us/tru...
4,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2519/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,41
26,deal,20
53,trade,20
36,gaza,17
20,market,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...,117
12,2025-07-28-16-27-53 +0000,nypost,Trump hails cease-fire win after halting trade...,https://nypost.com/2025/07/28/us-news/trump-ha...,104
25,2025-07-28-16-06-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...,102
2,2025-07-28-16-54-00 +0000,wsj,Stock Market Today: Euro Drops After Trump Str...,https://www.wsj.com/livecoverage/stock-market-...,101
196,2025-07-27-20-44-00 +0000,wsj,President Trump said he reached a trade agreem...,https://www.wsj.com/economy/trade/trump-says-h...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,117,2025-07-28-14-42-00 +0000,wsj,Trump said the U.S. reached a trade deal with ...,https://www.wsj.com/economy/trade/european-uni...
25,61,2025-07-28-16-06-00 +0000,wsj,President Trump said he would give Russia’s Vl...,https://www.wsj.com/politics/trump-losing-pati...
86,49,2025-07-28-13-14-04 +0000,nypost,Starving Palestinians swarm aid trucks in Gaza...,https://nypost.com/2025/07/28/world-news/pales...
2,40,2025-07-28-16-54-00 +0000,wsj,Stock Market Today: Euro Drops After Trump Str...,https://www.wsj.com/livecoverage/stock-market-...
14,39,2025-07-28-16-24-29 +0000,nyt,Thailand and Cambodia Agree to Halt Fighting T...,https://www.nytimes.com/2025/07/28/world/asia/...
4,37,2025-07-28-16-50-00 +0000,wsj,Business leaders on both sides of the Atlantic...,https://www.wsj.com/economy/trade/companies-we...
55,34,2025-07-28-15-04-00 +0000,wsj,Former Gov. Roy Cooper of North Carolina launc...,https://www.wsj.com/politics/elections/roy-coo...
108,27,2025-07-28-11-00-00 +0000,wsj,Most Americans support President Trump’s immig...,https://www.wsj.com/politics/policy/trump-immi...
74,24,2025-07-28-14-00-00 +0000,startribune,"Ramstad: Under investigation, UnitedHealth los...",https://www.startribune.com/ramstad-under-inve...
65,24,2025-07-28-14-38-14 +0000,nyt,3 Dead After Small Plane Crashes Off Californi...,https://www.nytimes.com/2025/07/27/us/plane-cr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
